In [6]:
import pymc3 as pm
import pandas as pd
import numpy as np
import arviz as az
import theano.tensor as tt

# 读取数据
ratio = pd.read_csv("data.csv")

# 将Location转换为分类变量
ratio['Location'] = ratio['Location'].astype('category')
print(ratio.head())
# 为PyMC3准备数据
coords = {"location": ratio.Location.cat.categories, "observation_idx": np.arange(ratio.shape[0])}
print(ratio.values)
data = {
    "Years": ratio.Years.values,
    "males": ratio.Males.values,
    "streams": ratio.Location.cat.codes.values,
    "N": ratio.shape[0]
}
print(data)

  Location.Name Location    Type  Years  Males
0    Bear River        1  Lentic      3      1
1    Bear River        1  Lentic      3      1
2    Bear River        1  Lentic      3      1
3    Bear River        1  Lentic      3      1
4    Bear River        1  Lentic      5      1
[['Bear River' 1 'Lentic' 3 1]
 ['Bear River' 1 'Lentic' 3 1]
 ['Bear River' 1 'Lentic' 3 1]
 ...
 ['White River' 8 'River' 1 1]
 ['White River' 8 'River' 1 1]
 ['White River' 8 'River' 1 0]]
{'Years': array([3, 3, 3, 3, 5, 4, 4, 4, 3, 3, 3, 5, 2, 3, 3, 3, 3, 2, 2, 3, 3, 3,
       2, 3, 5, 2, 3, 3, 3, 3, 1, 4, 5, 4, 4, 4, 4, 3, 4, 4, 4, 4, 5, 4,
       4, 4, 2, 2, 4, 4, 2, 3, 2, 4, 2, 3, 5, 2, 3, 2, 2, 2, 2, 3, 3, 2,
       2, 2, 3, 2, 3, 2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2,
       2, 2, 3, 2, 2, 2, 2, 1, 2, 2, 1, 1, 2, 2, 1, 0, 0, 0, 0, 4, 3, 3,
       5, 3, 4, 4, 4, 3, 4, 4, 3, 3, 3, 5, 3, 2, 3, 4, 3, 3, 3, 3, 2, 3,
       2, 3, 3, 3, 3, 2, 2, 4, 2, 2, 3, 2, 2, 2, 3, 2, 3, 3, 3, 2, 2, 3,
     

In [5]:
# 模型
with pm.Model(coords=coords) as hierarchical_model:
    # 组节点的超先验
    a = pm.Normal('a', mu=0, sd=10)
    b = pm.Normal('b', mu=0, sd=10)

    # 个体位置截距/斜率的先验
    a_location = pm.Normal('a_location', mu=a, sd=10, dims="location")
    b_location = pm.Normal('b_location', mu=b, sd=10, dims="location")
    p = pm.Deterministic('p',
                         pm.math.invlogit(a_location[data['streams']] + b_location[data['streams']] * data['Years']),
                         dims="observation_idx")
    print(p.tag.test_value)
    # 观测的似然性分布
    males_obs = pm.Bernoulli('males_obs', p=p[data['observation_idx']], observed=data['males'])

    # 推断：使用NUTS采样抽取2000个后验样本
    trace = pm.sample(draws=2000, tune=1000, chains=3, target_accept=0.95)

[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.

KeyError: 'observation_idx'

In [ ]:
# 检查收敛性
az.plot_trace(trace)
az.summary(trace)

# 后验预测检查
post_pred = pm.sample_posterior_predictive(trace, model=hierarchical_model)

# 计算不同年份时lentic和lotic之间的对数几率差异
lentic_idx = ratio[ratio.Location == 'lentic'].Location.cat.codes.unique()[0]
lotic_idx = ratio[ratio.Location == 'lotic'].Location.cat.codes.unique()[0]


In [ ]:
for year in range(7):
    lentic_log_odds = np.mean(trace['a_location'][:, lentic_idx] + trace['b_location'][:, lentic_idx] * year)
    lotic_log_odds = np.mean(trace['a_location'][:, lotic_idx] + trace['b_location'][:, lotic_idx] * year)
    print(f"年份 {year}, Lentic的对数几率: {lentic_log_odds}, Lotic的对数几率: {lotic_log_odds}")


In [ ]:
# 计算在lentic与stream环境中男性概率较大的概率
def calc_probability_comparison(trace, lentic_idx, lotic_idx, year):
    lentic_log_odds = trace['a_location'][:, lentic_idx] + trace['b_location'][:, lentic_idx] * year
    lotic_log_odds = trace['a_location'][:, lotic_idx] + trace['b_location'][:, lotic_idx] * year
    probabilities = 1 / (1 + np.exp(-lentic_log_odds)) > 1 / (1 + np.exp(-lotic_log_odds))
    return np.mean(probabilities)


In [ ]:
for year in range(7):
    probability = calc_probability_comparison(trace, lentic_idx, lotic_idx, year)
    print(f"年份 {year}, 在Lentic中男性概率较大的概率: {probability}")
